In [2]:
import scanpy as sc
import stlearn as st
import numpy as np
import csv
import pickle
from scipy import sparse
import scipy.io as sio
import matplotlib
matplotlib.use('Agg')
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, to_hex, rgb2hex
from typing import List
import qnorm
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from collections import defaultdict
import pandas as pd
import gzip
from kneed import KneeLocator
import copy 
#sys.path.append("/home/gw/code/utility/altairThemes/")
#if True:  # In order to bypass isort when saving
#    import altairThemes
#import altairThemes
import altair as alt
import argparse
import scipy.stats

spot_diameter = 89.43 #pixels

In [3]:
import sys
sys.path.append("/Users/victoriagao/Documents/MSc/Schwartz_lab/altair-themes/")
if True:  # In order to bypass isort when saving
    import altairThemes

In [4]:
def readCsv(x):
  """Parse file."""
  #colNames = ["method", "benchmark", "start", "end", "time", "memory"]
  df = pd.read_csv(x, sep=",")

  return df

def preprocessDf(df):
  """Transform ligand and receptor columns."""
  df["ligand-receptor"] = df["ligand"] + '-' + df["receptor"]
  df["component"] = df["component"] #.astype(str).str.zfill(2)

  return df


def plot(df):
  set1 = altairThemes.get_colour_scheme("Set1", len(df["component"].unique()))
  set1[0] = '#000000'
  base = alt.Chart(df).mark_bar().encode(
            x=alt.X("ligand-receptor:N", axis=alt.Axis(labelAngle=45), sort='-y'),
            y=alt.Y("count()"),
            color=alt.Color("component:N", scale = alt.Scale(range=set1)),
            order=alt.Order("component:N", sort="ascending"),
            tooltip=["component"]
        )
  p = base

  return p

In [5]:
##################### Below is all input #############################

In [6]:
data_name = 'PDAC_64630'

In [7]:
# read filtered matrix
adata_h5 = st.Read10X(path="/Users/victoriagao/local_docs/NEST/input", count_file='filtered_feature_bc_matrix.h5')
adata_h5


/Users/victoriagao/anaconda3/envs/NEST_try/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/victoriagao/anaconda3/envs/NEST_try/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 1406 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [8]:
adata_h5.X

<1406x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 2345152 stored elements in Compressed Sparse Row format>

In [9]:
# filter out the genes that are not expressed in any cell
sc.pp.filter_genes(adata_h5, min_cells=1)
adata_h5

AnnData object with n_obs × n_vars = 1406 × 19523
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'spatial'
    obsm: 'spatial'

In [10]:
# get gene identifier - names
gene_ids = list(adata_h5.var_names)
gene_ids

['AL627309.1',
 'AL627309.5',
 'AP006222.2',
 'LINC01409',
 'LINC01128',
 'LINC00115',
 'FAM41C',
 'LINC02593',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'AL645608.7',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'LINC01342',
 'AL390719.2',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'AL162741.1',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20-AS1',
 'MRPL20',
 'AL391244.2',
 'ANKRD65',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'AL645728.1',
 'FNDC10',
 'AL691432.2',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'SLC35E2A',
 'NADK',
 'GNB1',
 'AL109917.1',
 'CALML6',
 'TMEM52',
 'AL391845.2',
 'GABRD',
 'PRKCZ',
 'PRKCZ-AS1',
 'FAAP20',
 'SKI',
 'AL590822.3',
 'MORN1',
 'AL589739.1',
 'AL513477.2',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'AL139246.5',
 'TNFRSF14-AS1',
 'TNFRSF14',
 'AL139246.3',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'AC

In [11]:
# access coordinates stored in adata_ha using the key spatial
coordinates = adata_h5.obsm['spatial']
coordinates

array([[ 9283, 12018],
       [11004, 10622],
       [ 8053,  6963],
       ...,
       [12040, 10617],
       [ 9801, 12115],
       [ 3230,  9762]])

In [12]:
# all x-position of cells
coordinates[:,0]
# all y-position of cells
coordinates[0,:]

array([ 9283, 12018])

In [13]:
# get cell identifier - barcode
cell_barcode = np.array(adata_h5.obs.index)
temp = adata_h5.X

In [14]:
cell_barcode

array(['AAACCGGGTAGGTACC-1', 'AAACCGTTCGTCCAGG-1', 'AAACGAGACGGTTGAT-1',
       ..., 'TTGTTTCACATCCAGG-1', 'TTGTTTCCATACAACT-1',
       'TTGTTTGTGTAAATTC-1'], dtype=object)

In [32]:
## get the cell vs gene matrix ##################
temp = qnorm.quantile_normalize(np.transpose(sparse.csr_matrix.toarray(temp)))
adata_X = np.transpose(temp)

cell_vs_gene = copy.deepcopy(adata_X)
## get the cell vs gene matrix ##################

In [126]:
##################### make cell metadata: barcode_info ###################################
 
i=0
barcode_serial = dict()
for cell_code in cell_barcode:
    barcode_serial[cell_code] = i
    i = i + 1

i=0
barcode_info=[]
for cell_code in cell_barcode:
    barcode_info.append([cell_code, coordinates[i,0],coordinates[i,1], 0]) # last entry will hold the component number later
    i=i+1
    

In [16]:
barcode_serial

{'AAACCGGGTAGGTACC-1': 0,
 'AAACCGTTCGTCCAGG-1': 1,
 'AAACGAGACGGTTGAT-1': 2,
 'AAACTGCTGGCTCCAA-1': 3,
 'AAAGACCCAAGTCGCG-1': 4,
 'AAAGGCTCTCGCGCCG-1': 5,
 'AAAGGGATGTAGCAAG-1': 6,
 'AAAGGGCAGCTTGAAT-1': 7,
 'AAAGTAGCATTGCTCA-1': 8,
 'AAAGTGTGATTTATCT-1': 9,
 'AAATACCTATAAGCAT-1': 10,
 'AAATAGGGTGCTATTG-1': 11,
 'AAATCGTGTACCACAA-1': 12,
 'AAATGGTCAATGTGCC-1': 13,
 'AAATTAACGGGTAGCT-1': 14,
 'AAATTGATAGTCCTTT-1': 15,
 'AAATTTGCGGGTGTGG-1': 16,
 'AACAACTGGTAGTTGC-1': 17,
 'AACAATTACTCTACGC-1': 18,
 'AACACACGCTCGCCGC-1': 19,
 'AACAGGATGGGCCGCG-1': 20,
 'AACCAAGACTTCTCTG-1': 21,
 'AACCCAGAGACGGAGA-1': 22,
 'AACCGCTAAGGGATGC-1': 23,
 'AACCTTTAAATACGGT-1': 24,
 'AACGATAATGCCGTAG-1': 25,
 'AACGCGGTCTCCAGCC-1': 26,
 'AACGCTGTTGCTGAAA-1': 27,
 'AACGGACGTACGTATA-1': 28,
 'AACGGCCATCTCCGGT-1': 29,
 'AACGTACTGTGGGTAC-1': 30,
 'AACGTCAGACTAGTGG-1': 31,
 'AACGTGCGAAAGTCTC-1': 32,
 'AACTAGGCTTGGGTGT-1': 33,
 'AACTCAAGTTAATTGC-1': 34,
 'AACTCTCAGTGTGCTC-1': 35,
 'AACTTGCCCGTATGCA-1': 36,
 'AAGACTGCA

In [127]:
# a list of lists of: barcode, coordinate x position, coordinate y position, 0
barcode_info

[['AAACCGGGTAGGTACC-1', 9283, 12018, 0],
 ['AAACCGTTCGTCCAGG-1', 11004, 10622, 0],
 ['AAACGAGACGGTTGAT-1', 8053, 6963, 0],
 ['AAACTGCTGGCTCCAA-1', 9785, 8146, 0],
 ['AAAGACCCAAGTCGCG-1', 3750, 10057, 0],
 ['AAAGGCTCTCGCGCCG-1', 11516, 9329, 0],
 ['AAAGGGATGTAGCAAG-1', 6161, 8658, 0],
 ['AAAGGGCAGCTTGAAT-1', 6176, 12230, 0],
 ['AAAGTAGCATTGCTCA-1', 10837, 12111, 0],
 ['AAAGTGTGATTTATCT-1', 9600, 5468, 0],
 ['AAATACCTATAAGCAT-1', 10129, 7946, 0],
 ['AAATAGGGTGCTATTG-1', 11161, 6949, 0],
 ['AAATCGTGTACCACAA-1', 9617, 9238, 0],
 ['AAATGGTCAATGTGCC-1', 7719, 9743, 0],
 ['AAATTAACGGGTAGCT-1', 7889, 9047, 0],
 ['AAATTGATAGTCCTTT-1', 11526, 11711, 0],
 ['AAATTTGCGGGTGTGG-1', 10144, 11518, 0],
 ['AACAACTGGTAGTTGC-1', 6860, 10639, 0],
 ['AACAATTACTCTACGC-1', 7040, 12325, 0],
 ['AACACACGCTCGCCGC-1', 10809, 5562, 0],
 ['AACAGGATGGGCCGCG-1', 7037, 11730, 0],
 ['AACCAAGACTTCTCTG-1', 8249, 12419, 0],
 ['AACCCAGAGACGGAGA-1', 4617, 10947, 0],
 ['AACCGCTAAGGGATGC-1', 9947, 5665, 0],
 ['AACCTTTAAATACGGT-

In [18]:
len(barcode_info)

1406

In [19]:
## load annotations ##

if data_name == 'PDAC_64630':
    pathologist_label_file='/Users/victoriagao/local_docs/NEST/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]
        

In [ ]:
##################### Above is all input #############################

### Read NEST output in csv format

In [283]:
filename_str = 'NEST_combined_output_PDAC_64630'+'.csv'
inputFile = '/Users/victoriagao/local_docs/NEST/output/'+filename_str
df = pd.read_csv(inputFile, sep=",")
csv_record_final = df.values.tolist()
df_column_names = list(df.columns)
csv_record_final = [df_column_names] + csv_record_final

component_list = dict()
for record_idx in range (1, len(csv_record_final)-1): #last entry is a dummy for histograms, so ignore it.
    record = csv_record_final[record_idx]
    i = record[6]
    j = record[7]
    component_label = record[5]
    barcode_info[i][3] = component_label 
    barcode_info[j][3] = component_label
    component_list[component_label] = ''

component_list[0] = ''
unique_component_count = len(component_list.keys())

In [161]:
csv_record_final

[['from_cell',
  'to_cell',
  'ligand',
  'receptor',
  'attention_score',
  'component',
  'from_id',
  'to_id'],
 ['ACGCGCTACACAGGGT-1',
  'AAACCGGGTAGGTACC-1',
  'LGALS3',
  'NPTN',
  0.9119191408157348,
  0,
  132,
  0],
 ['TCTTACTTATGCCTCT-1',
  'AAAGTGTGATTTATCT-1',
  'FN1',
  'RPSA',
  0.9299875378608704,
  0,
  1228,
  9],
 ['AGTCTCACAAGACTAC-1',
  'AAATTGATAGTCCTTT-1',
  'PTPRF',
  'RACK1',
  0.9115079283714296,
  0,
  228,
  15],
 ['GTGGGCTTAGACACAC-1',
  'AACAACTGGTAGTTGC-1',
  'FN1',
  'RPSA',
  0.9233949303627014,
  0,
  992,
  17],
 ['CCACAGTACCCATCCT-1',
  'AACCGCTAAGGGATGC-1',
  'FN1',
  'RPSA',
  0.9293025016784668,
  0,
  425,
  23],
 ['AATAACACTAGAACAA-1',
  'AACTCTCAGTGTGCTC-1',
  'INS',
  'HLA',
  0.9678001403808594,
  0,
  62,
  35],
 ['GAATTTCTCGCTGCAG-1',
  'AACTTGCCCGTATGCA-1',
  'FN1',
  'RPSA',
  0.929611313343048,
  0,
  696,
  36],
 ['GCGATTGTTAACGTTA-1',
  'AAGAGCTCTTTATCGG-1',
  'PTPRF',
  'RACK1',
  0.9112038850784302,
  0,
  802,
  39],
 ['CTTTCTGTGCGGG

#### Specify csv_record_final for components/regions

In [282]:
# by specifying regions we DO NOT WANT TO PLOT ####
# region_of_interest = [...] 

# i.e. plot no stroma regions: tumor-tumor, tumor-stroma, stroma-tumor
# for record_idx in range (1, len(csv_record_final)-1): #last entry is a dummy for histograms, ignore.
#     if barcode_type[csv_record_final[record_idx][0]] != 'tumor' and barcode_type[csv_record_final[record_idx][1]] != 'tumor':csv_record_final[record_idx][5]= 0 # This will exclude all tumor-tumor and stroma-stroma interactions

##### i.e. plot only tumor-tumor regions
# for record_idx in range (1, len(csv_record_final)-1): #last entry is a dummy for histograms, ignore.
#     if barcode_type[csv_record_final[record_idx][0]] != 'tumor' or barcode_type[csv_record_final[record_idx][1]] != 'tumor':csv_record_final[record_idx][5]= 0 # This will exclude all tumor-tumor and stroma-stroma interactions

##### i.e. plot only for cell component 13
# for record_idx in range (1, len(csv_record_final)-1): 
#     if csv_record_final[record_idx][5] != 13:csv_record_final[record_idx][5]= 0 

##### i.e. plot only for CCC between one component and everything else, i.e. set ligand as component 13 and receptor as everything else but 13

##### i.e. plot only within a certain region by setting a boundary xy coordinate
x_boundary = (10000, 13000)
y_boundary = (5000, 9000)
barcode_info_dict = {item[0]: [item[1], item[2]] for item in barcode_info}

# components outside x-axis range 
for record_idx in range (1, len(csv_record_final)-1): 
    if barcode_info_dict[csv_record_final[record_idx][0]][0] < x_boundary[0] or barcode_info_dict[csv_record_final[record_idx][0]][0] > x_boundary[1]:csv_record_final[record_idx][5]= 0
for record_idx in range (1, len(csv_record_final)-1): 
    if barcode_info_dict[csv_record_final[record_idx][1]][0] < x_boundary[0] or barcode_info_dict[csv_record_final[record_idx][1]][0] > x_boundary[1]:csv_record_final[record_idx][5]= 0

# components outside y-axis range 
for record_idx in range (1, len(csv_record_final)-1): 
    if barcode_info_dict[csv_record_final[record_idx][0]][1] < y_boundary[0] or barcode_info_dict[csv_record_final[record_idx][0]][1] > y_boundary[1]:csv_record_final[record_idx][5]= 0
for record_idx in range (1, len(csv_record_final)-1): 
    if barcode_info_dict[csv_record_final[record_idx][1]][1] < y_boundary[0] or barcode_info_dict[csv_record_final[record_idx][1]][1] > y_boundary[1]:csv_record_final[record_idx][5]= 0




################change csv_record_final for specific components/regions only ##############

In [163]:
csv_record_final

[['from_cell',
  'to_cell',
  'ligand',
  'receptor',
  'attention_score',
  'component',
  'from_id',
  'to_id'],
 ['ACGCGCTACACAGGGT-1',
  'AAACCGGGTAGGTACC-1',
  'LGALS3',
  'NPTN',
  0.9119191408157348,
  0,
  132,
  0],
 ['TCTTACTTATGCCTCT-1',
  'AAAGTGTGATTTATCT-1',
  'FN1',
  'RPSA',
  0.9299875378608704,
  0,
  1228,
  9],
 ['AGTCTCACAAGACTAC-1',
  'AAATTGATAGTCCTTT-1',
  'PTPRF',
  'RACK1',
  0.9115079283714296,
  0,
  228,
  15],
 ['GTGGGCTTAGACACAC-1',
  'AACAACTGGTAGTTGC-1',
  'FN1',
  'RPSA',
  0.9233949303627014,
  0,
  992,
  17],
 ['CCACAGTACCCATCCT-1',
  'AACCGCTAAGGGATGC-1',
  'FN1',
  'RPSA',
  0.9293025016784668,
  0,
  425,
  23],
 ['AATAACACTAGAACAA-1',
  'AACTCTCAGTGTGCTC-1',
  'INS',
  'HLA',
  0.9678001403808594,
  0,
  62,
  35],
 ['GAATTTCTCGCTGCAG-1',
  'AACTTGCCCGTATGCA-1',
  'FN1',
  'RPSA',
  0.929611313343048,
  0,
  696,
  36],
 ['GCGATTGTTAACGTTA-1',
  'AAGAGCTCTTTATCGG-1',
  'PTPRF',
  'RACK1',
  0.9112038850784302,
  0,
  802,
  39],
 ['CTTTCTGTGCGGG

In [57]:
# check new specified data
see_column_values = [sublist[5] for sublist in csv_record_final]
see_column_values

['component',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 21,
 22,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 13,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 13,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 13,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 21,
 0,
 0,
 0,
 68,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 13,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 80,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 

In [58]:
csv_record_final[1]

['ACGCGCTACACAGGGT-1',
 'AAACCGGGTAGGTACC-1',
 'LGALS3',
 'NPTN',
 0.9119191408157348,
 0,
 132,
 0]

#### Create active_spot file: dictionary of spots participating in CCC 

In [239]:
active_spot = defaultdict(list)

for record_idx in range (1, len(csv_record_final)-1): #last entry is a dummy for histograms, so ignore it.
    record = csv_record_final[record_idx]
    
    # ligand
    i = record[6] # correspond to the column number in the csv file 
    pathology_label = barcode_type[barcode_info[i][0]] # get the correponding tumor or stroma type
    component_label = record[5]
    X = barcode_info[i][1] # barcode_info file was created earlier
    Y = -barcode_info[i][2]
    opacity = record[4] # set opacity based on attention_score obtained by NEST
    active_spot[i].append([pathology_label, component_label, X, Y, opacity])
    
    # receptor
    j = record[7] 
    pathology_label = barcode_type[barcode_info[j][0]]
    component_label = record[5]
    X = barcode_info[j][1]
    Y = -barcode_info[j][2]
    opacity = record[4]   
    active_spot[j].append([pathology_label, component_label, X, Y, opacity])

In [53]:
active_spot

defaultdict(list,
            {132: [1, 0, 9109, -11721, 0.9119191408157348],
             0: [1, 0, 9283, -12018, 0.9119191408157348],
             1228: [0, 0, 9773, -5566, 0.9301460146903991],
             9: [0, 0, 9600, -5468, 0.9299875378608704],
             228: [0, 0, 11354, -11811, 0.912053906917572],
             15: [0, 0, 11526, -11711, 0.9115079283714296],
             992: [0, 0, 7034, -10936, 0.9233949303627014],
             17: [0, 0, 6860, -10639, 0.9233949303627014],
             425: [0, 0, 10292, -5663, 0.9293025016784668],
             23: [0, 0, 9947, -5665, 0.9293025016784668],
             62: [0, 0, 11005, -11019, 0.9678001403808594],
             35: [0, 0, 11177, -10720, 0.9678001403808594],
             696: [0, 0, 6865, -11830, 0.9305546641349792],
             36: [1, 0, 7039, -12127, 0.929611313343048],
             802: [0, 0, 8399, -7160, 0.9119569540023804],
             39: [0, 0, 8054, -7161, 0.9112038850784302],
             667: [0, 21, 11339, -8

In [52]:
active_spot[132]

[1, 0, 9109, -11721, 0.9119191408157348]

#### Color the spots in the plot with opacity = attention score 

In [240]:
opacity_list = []
for i in active_spot:
    sum_opacity = []
    for edges in active_spot[i]:
        sum_opacity.append(edges[4])
        
    # avg_opacity = np.mean(sum_opacity)
    max_opacity = np.max(sum_opacity)
    opacity_list.append(max_opacity) 
    active_spot[i]=[active_spot[i][0][0], active_spot[i][0][1], active_spot[i][0][2], active_spot[i][0][3], max_opacity]

min_opacity = np.min(opacity_list)
max_opacity = np.max(opacity_list)
min_opacity = min_opacity - 5

### altair plot 

In [241]:
# put everything into pandas df for altair plotting
data_list=dict()
data_list['pathology_label']=[]
data_list['component_label']=[]
data_list['X']=[]
data_list['Y']=[]   
data_list['opacity']=[] 

for i in range (0, len(barcode_info)):        
    if i in active_spot:
        data_list['pathology_label'].append(active_spot[i][0])
        data_list['component_label'].append(active_spot[i][1])
        data_list['X'].append(active_spot[i][2])
        data_list['Y'].append(active_spot[i][3])
        data_list['opacity'].append((active_spot[i][4]-min_opacity)/(max_opacity-min_opacity))
        
    else:
        data_list['pathology_label'].append(barcode_type[barcode_info[i][0]])
        data_list['component_label'].append(0)
        data_list['X'].append(barcode_info[i][1])
        data_list['Y'].append(-barcode_info[i][2])
        data_list['opacity'].append(0.1)

In [242]:
id_label= len(list(set(data_list['component_label'])))

data_list_pd = pd.DataFrame(data_list) # turn 

set1 = altairThemes.get_colour_scheme("Set1", id_label)
set1[0] = '#000000'
chart = alt.Chart(data_list_pd).mark_point(filled=True, opacity = 1).encode(
    alt.X('X', scale=alt.Scale(zero=False)),
    alt.Y('Y', scale=alt.Scale(zero=False)),
    shape = alt.Shape('pathology_label:N'), #shape = "pathology_label",
    color=alt.Color('component_label:N', scale=alt.Scale(range=set1)),
    #opacity=alt.Opacity('opacity:N'), #"opacity",
    tooltip=['component_label'] #,'opacity'
)#.configure_legend(labelFontSize=6, symbolLimit=50)

In [243]:
len(list(set(data_list['component_label'])))

87

In [237]:
save_path = '/Users/victoriagao/local_docs/NEST/output/visualization_html'
chart.save(save_path+'altair_plot_test.html')

###  Histogram plotting 

In [292]:
alt.themes.register("publishTheme", altairThemes.publishTheme)
alt.themes.enable("publishTheme")

## Defult whole sample 
# df = readCsv("/Users/victoriagao/local_docs/NEST/output/NEST_combined_output_PDAC_64630.csv")
# df = preprocessDf(df)
# outPathRoot = inputFile.split('.')[0]

## Plot only component 13
# columns = ['from_cell','to_cell','ligand','receptor','attention_score','component','from_id','to_id'] # make csv_record_final into a dataframe
# df = pd.DataFrame(csv_record_final, columns=columns)
# df.loc[df['component'] == 13] = df_13
# df_13 = preprocessDf(df_13)

## Plot only specified region of sample 
x_boundary = (11000, 13000)
y_boundary = (12000, 13000)

filtered_cells = []
plot_data = []

# Filter each cell by their coordinates
for cell_id, (x, y) in barcode_info_dict.items():
    if x_boundary[0] <= x <= x_boundary[1]:
        filtered_cells.append((cell_id, (x, y)))
        
# For all filtered cells, match to csv_record_final - interaction pairs
# if either the ligand or receptor is a cell within range, the interaction will be included
for cell_id, _ in filtered_cells:
    for row in csv_record_final:
        if row[0] == cell_id or row[1] == cell_id: 
            plot_data.append(row)

for row in plot_data:
    print(row)

# assume csv_record_final is already filtered
columns = ['from_cell','to_cell','ligand','receptor','attention_score','component','from_id','to_id'] # make csv_record_final into a dataframe
df = pd.DataFrame(plot_data, columns=columns)
df = preprocessDf(df)



['AAAGGCTCTCGCGCCG-1', 'CTGGCTGGTTGTCAGT-1', 'INHBA', 'ENG', 0.909631049633026, 3, 5, 634]
['AAAGGCTCTCGCGCCG-1', 'TGCCAATGGGTACTCT-1', 'INS', 'INSR', 0.9306722819805144, 3, 5, 1258]
['AGTCTCACAAGACTAC-1', 'AAATTGATAGTCCTTT-1', 'PTPRF', 'RACK1', 0.9115079283714296, 10, 228, 15]
['CCATTTCTACCTATTA-1', 'AACCTTTAAATACGGT-1', 'TNFSF13', 'TNFRSF14', 0.7139014482498169, 13, 441, 24]
['CCATTTCTACCTATTA-1', 'AACCTTTAAATACGGT-1', 'PTPRF', 'INSR', 0.7050918102264404, 13, 441, 24]
['CCATTTCTACCTATTA-1', 'AACCTTTAAATACGGT-1', 'PTPRF', 'RACK1', 0.7051778674125672, 13, 441, 24]
['TTCACTCGAGCACCTA-1', 'AACCTTTAAATACGGT-1', 'PTPRF', 'INSR', 0.7063581943511963, 13, 1341, 24]
['TTCACTCGAGCACCTA-1', 'AACCTTTAAATACGGT-1', 'PTPRF', 'RACK1', 0.7064443349838256, 13, 1341, 24]
['AACCTTTAAATACGGT-1', 'ATTCACTGATGTTGGA-1', 'TIMP1', 'LRP1', 0.7017103552818298, 13, 24, 316]
['AACCTTTAAATACGGT-1', 'TTCACTCGAGCACCTA-1', 'HMGB1', 'AGTRAP', 0.7026095509529113, 13, 24, 1341]
['CTGCACTCCAGTACAG-1', 'AACGATAATGCCGTAG-1'

In [293]:
# plot df
p = plot(df)
outPath = '/Users/victoriagao/local_docs/NEST/output/new_hist.html'
p.save(outPath)

In [288]:
df

,from_cell,to_cell,ligand,receptor,attention_score,component,from_id,to_id,ligand-receptor
0,AAAGGCTCTCGCGCCG-1,CTGGCTGGTTGTCAGT-1,INHBA,ENG,0.909631,3,5,634,INHBA-ENG
1,AAAGGCTCTCGCGCCG-1,TGCCAATGGGTACTCT-1,INS,INSR,0.930672,3,5,1258,INS-INSR
2,AAAGTAGCATTGCTCA-1,ACCTAAGTACCTTTCA-1,RPS19,RPSA,0.929789,5,8,124,RPS19-RPSA
3,AAATACCTATAAGCAT-1,GGCAAGGCGAAATAGC-1,WNT5A,TFRC,0.944570,7,10,861,WNT5A-TFRC
4,GTATCTTTCATAACCA-1,AAATACCTATAAGCAT-1,TGFB1,ITGB5,0.835687,7,950,10,TGFB1-ITGB5
...,...,...,...,...,...,...,...,...,...
901,TTGTGGTAGGAGGGAT-1,TCAACGCAGGAAATAA-1,THBS1,LRP5,0.911078,79,1397,1128,THBS1-LRP5
902,GTAGAGGGAGACAAGT-1,TTGTGGTAGGAGGGAT-1,TGFB3,SDC4,0.868089,79,943,1397,TGFB3-SDC4
903,AATAGTCCGTCCCGAC-1,TTGTGTATGCCACCAA-1,LGALS3,NPTN,0.911787,24,65,1398,LGALS3-NPTN
904,TTGTGTTTCCCGAAAG-1,GTGAGTGGTACAACGC-1,FN1,RPSA,0.928436,3,1399,978,FN1-RPSA


###  Network Plot

In [220]:
# # find the corresponding component number to each spot
# barcodes_in_barcode_info = [sublist[0] for sublist in barcode_info]
# corresponding_component_num = [item[5] for item in csv_record_final if (item[0] in barcodes_in_barcode_info or item[1] in barcodes_in_barcode_info)]

# # assign the corresponding component number to each spot
# for sublist, new_value in zip(barcode_info, corresponding_component_num):
#     sublist[-1] = new_value


In [274]:
barcode_info

[['AAACCGGGTAGGTACC-1', 9283, 12018, 2],
 ['AAACCGTTCGTCCAGG-1', 11004, 10622, 6],
 ['AAACGAGACGGTTGAT-1', 8053, 6963, 10],
 ['AAACTGCTGGCTCCAA-1', 9785, 8146, 11],
 ['AAAGACCCAAGTCGCG-1', 3750, 10057, 16],
 ['AAAGGCTCTCGCGCCG-1', 11516, 9329, 3],
 ['AAAGGGATGTAGCAAG-1', 6161, 8658, 4],
 ['AAAGGGCAGCTTGAAT-1', 6176, 12230, 20],
 ['AAAGTAGCATTGCTCA-1', 10837, 12111, 5],
 ['AAAGTGTGATTTATCT-1', 9600, 5468, 6],
 ['AAATACCTATAAGCAT-1', 10129, 7946, 7],
 ['AAATAGGGTGCTATTG-1', 11161, 6949, 23],
 ['AAATCGTGTACCACAA-1', 9617, 9238, 8],
 ['AAATGGTCAATGTGCC-1', 7719, 9743, 9],
 ['AAATTAACGGGTAGCT-1', 7889, 9047, 18],
 ['AAATTGATAGTCCTTT-1', 11526, 11711, 10],
 ['AAATTTGCGGGTGTGG-1', 10144, 11518, 27],
 ['AACAACTGGTAGTTGC-1', 6860, 10639, 11],
 ['AACAATTACTCTACGC-1', 7040, 12325, 12],
 ['AACACACGCTCGCCGC-1', 10809, 5562, 13],
 ['AACAGGATGGGCCGCG-1', 7037, 11730, 14],
 ['AACCAAGACTTCTCTG-1', 8249, 12419, 15],
 ['AACCCAGAGACGGAGA-1', 4617, 10947, 5],
 ['AACCGCTAAGGGATGC-1', 9947, 5665, 16],
 ['AAC

In [276]:
# set1 = altairThemes.get_colour_scheme("Set1", id_label)
set1 = altairThemes.get_colour_scheme("Set1", id_label)
colors = set1
colors[0] = '#000000'
ids = []
x_index=[]
y_index=[]
colors_point = []
for i in range (0, len(barcode_info)):    
    ids.append(i)
    x_index.append(barcode_info[i][1])
    y_index.append(barcode_info[i][2])
    color_index = barcode_info[i][3]
    # print(colors[color_index])
    colors_point.append(colors[color_index]) 
  
max_x = np.max(x_index)
max_y = np.max(y_index)

In [249]:
colors_point

['#c92a34',
 '#884f6f',
 '#4775aa',
 '#377eb8',
 '#419584',
 '#48a365',
 '#3d8c99',
 '#4aa85a',
 '#4cac50',
 '#50ab4d',
 '#57a256',
 '#57a256',
 '#4775aa',
 '#4775aa',
 '#469e6f',
 '#737e77',
 '#737e77',
 '#7a7480',
 '#3b87a3',
 '#984661',
 '#984661',
 '#984661',
 '#984661',
 '#984661',
 '#b93343',
 '#b93343',
 '#3b87a3',
 '#a95688',
 '#a95688',
 '#b35b78',
 '#3b87a3',
 '#c66459',
 '#a05297',
 '#3983ad',
 '#3983ad',
 '#3b87a3',
 '#3b87a3',
 '#449a79',
 '#449a79',
 '#3f918e',
 '#737e77',
 '#4aa85a',
 '#469e6f',
 '#469e6f',
 '#469e6f',
 '#4775aa',
 '#449a79',
 '#449a79',
 '#c92a34',
 '#b35b78',
 '#3983ad',
 '#f77b0d',
 '#ff8904',
 '#ffa10e',
 '#469e6f',
 '#a95688',
 '#884f6f',
 '#ffb917',
 '#ffc51c',
 '#ffd121',
 '#ffd121',
 '#ffd121',
 '#ffe92a',
 '#ff9509',
 '#fff52f',
 '#a83d52',
 '#a83d52',
 '#3983ad',
 '#3983ad',
 '#3983ad',
 '#3983ad',
 '#4775aa',
 '#4775aa',
 '#449a79',
 '#b35b78',
 '#f5ec32',
 '#469e6f',
 '#469e6f',
 '#3b87a3',
 '#48a365',
 '#737e77',
 '#e4cc30',
 '#a95688',
 '#5

In [250]:
set1

['#000000',
 '#d92026',
 '#c92a34',
 '#b93343',
 '#a83d52',
 '#984661',
 '#884f6f',
 '#77597e',
 '#67628d',
 '#576c9b',
 '#4775aa',
 '#377eb8',
 '#3983ad',
 '#3b87a3',
 '#3d8c99',
 '#3f918e',
 '#419584',
 '#449a79',
 '#469e6f',
 '#48a365',
 '#4aa85a',
 '#4cac50',
 '#50ab4d',
 '#57a256',
 '#5e995e',
 '#659067',
 '#6c876f',
 '#737e77',
 '#7a7480',
 '#816b88',
 '#886291',
 '#8f5999',
 '#9750a1',
 '#a05297',
 '#a95688',
 '#b35b78',
 '#bd5f69',
 '#c66459',
 '#d0694a',
 '#da6d3b',
 '#e4722b',
 '#ed771c',
 '#f77b0d',
 '#ff8101',
 '#ff8904',
 '#ff9509',
 '#ffa10e',
 '#ffad12',
 '#ffb917',
 '#ffc51c',
 '#ffd121',
 '#ffdd26',
 '#ffe92a',
 '#fff52f',
 '#fdfc33',
 '#f5ec32',
 '#eddc31',
 '#e4cc30',
 '#dcbc2f',
 '#d3ac2e',
 '#cb9c2d',
 '#c38c2c',
 '#ba7c2b',
 '#b26d29',
 '#a95d28',
 '#aa5830',
 '#b25c3f',
 '#ba604d',
 '#c1655b',
 '#c96969',
 '#d16d77',
 '#d87186',
 '#e07594',
 '#e779a2',
 '#ef7db0',
 '#f781be',
 '#ef83bc',
 '#e685b8',
 '#dd88b4',
 '#d48ab1',
 '#cb8cad',
 '#c28eaa',
 '#b991a6',
 '#b

#### Visualize the actual communication

In [277]:
import matplotlib
from pyvis.network import Network
import networkx as nx

barcode_type=dict()
for i in range (1, len(pathologist_label)):
    if 'tumor'in pathologist_label[i][1]: #'Tumour':
        barcode_type[pathologist_label[i][0]] = 1
    else:
        barcode_type[pathologist_label[i][0]] = 0
    '''
    elif pathologist_label[i][1] == 'stroma_deserted':
        barcode_type[pathologist_label[i][0]] = 0
    elif pathologist_label[i][1] =='acinar_reactive':
        barcode_type[pathologist_label[i][0]] = 2
    else:
        barcode_type[pathologist_label[i][0]] = 'zero' #0
    '''
g = nx.MultiDiGraph(directed=True) #nx.Graph()
for i in range (0, len(barcode_info)):
    label_str =  str(i)+'_c:'+str(barcode_info[i][3])+'_'
    #if barcode_type[barcode_info[i][0]] == 'zero':
    #    continue
    if barcode_type[barcode_info[i][0]] == 0: #stroma
        marker_size = 'circle'
        label_str = label_str + 'stroma'
    elif barcode_type[barcode_info[i][0]] == 1: #tumor
        marker_size = 'box'
        label_str = label_str + 'tumor'
    else:
        marker_size = 'ellipse'
        label_str = label_str + 'acinar_reactive'
	
    g.add_node(int(ids[i]), x=int(x_index[i]), y=int(y_index[i]), label = label_str, pos = str(x_index[i])+","+str(-y_index[i])+" !", physics=False, shape = marker_size, color=matplotlib.colors.rgb2hex(colors_point[i]))
    # print(colors_point[i])

nt = Network( directed=True, height='1000px', width='100%') #"500px", "500px",, filter_menu=True



In [278]:
# count_edges = 0
# for k in range (1, len(csv_record_final)):
#     i = csv_record_final[k][6]
#     j = csv_record_final[k][7]    
#     ligand = csv_record_final[k][2]
#     receptor = csv_record_final[k][3]
#     title_str =  "L:"+ligand+", R:"+receptor
#     edge_score = csv_record_final[k][4]
#     g.add_edge(int(i), int(j), label = title_str, value=np.float64(edge_score), color=colors_point[i] ) 
#     count_edges = count_edges + 1

for interaction in csv_record_final[1:]:
    ligand_id = interaction[6]
    receptor_id = interaction[7]
    ligand_name = interaction[2]
    receptor_name = interaction[3]
    edge_score = interaction[4]
    # print(type(interaction[4]))
    interaction_label = "L:"+ligand_name+", R:"+receptor_name

    g.add_edge(ligand_id, receptor_id, interaction_label=interaction_label, value=np.float64(edge_score), color=colors_point[ligand_id])
    # print(colors_point[ligand_id])


In [279]:
nt.from_nx(g)
# nt.show('mygraph.html')
nt.save_graph('mygraph.html')
import os
os.system('cp mygraph.html /Users/victoriagao/local_docs/NEST/mygraph.html')
# cp mygraph.html /Users/victoriagao/local_docs/NEST/output/mygraph.html

# from networkx.drawing.nx_agraph import write_dot
# write_dot(g, "/cluster/home/t116508uhn/64630/test_interactive.dot")

0

In [152]:
#### Checking properties of network
# if nt.nodes and nt.edges:
#     print("Network graph has nodes and edges.")
# else:
#     print("Network graph is empty or not properly created.")

Network graph has nodes and edges.


In [191]:
nt

<class 'pyvis.network.Network'> |N|=1408 |E|=1,094

In [ ]:
list(g.nodes)

In [ ]:
list(g.edges)